python3 -m pip install -U git+https://github.com/adefossez/demucs#egg=demucs

In [1]:
import demucs.api
import soundfile as sf
import torchaudio
import torch
import scipy.io as sio
import tensorflow as tf
import scipy
import numpy as np
import csv
import pandas as pd
import time

# import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile

# import tensorflow_hub as hub
# import kagglehub
# path = kagglehub.model_download("google/yamnet/tensorFlow2/yamnet")

/Users/andrewchang/opt/anaconda3/envs/demucs/lib/python3.9/site-packages/demucs/api.py:27: UserWarning: TorchAudio's global backend is now deprecated. Please enable distpatcher by setting `TORCHAUDIO_USE_BACKEND_DISPATCHER=1`, and specify backend when calling load/info/save function.
  import torchaudio as ta
2024-04-08 11:40:16.415372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df_meta = pd.read_csv('STM_output/STM_metaData/metaData_fma_large.csv', index_col=0)
df_meta.head()

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,corpus,mat_filename,speaker/artist,genre,gender
0,data/musicCorp/fma_large/083/083813.mp3,83813,NaN,NaN,1,1234800,44.1,28,0.032880,28,fma_large,STM_output/MATs/music_mat_wl4_fma_large/083813...,Brandon Ross,"['Jazz', 'Jazz: Vocal']",NaN
1,data/musicCorp/fma_large/034/034059.mp3,34059,NaN,NaN,1,1234800,44.1,28,0.034807,28,fma_large,STM_output/MATs/music_mat_wl4_fma_large/034059...,Sound Of Ground,"['Psych-Rock', 'Blues', 'Rock', 'Loud-Rock']",NaN
2,data/musicCorp/fma_large/092/092378.mp3,92378,NaN,NaN,1,1234800,44.1,28,0.020907,28,fma_large,STM_output/MATs/music_mat_wl4_fma_large/092378...,M-PeX,"['Jazz', 'Hip-Hop', 'Electronic']",NaN
3,data/musicCorp/fma_large/013/013935.mp3,13935,en,NaN,1,1234800,44.1,28,0.006553,28,fma_large,STM_output/MATs/music_mat_wl4_fma_large/013935...,Milton Cross,"['Instrumental', 'Sound Collage', 'Ambient', '...",NaN
4,data/musicCorp/fma_large/148/148436.mp3,148436,NaN,NaN,1,1234800,44.1,28,0.006735,28,fma_large,STM_output/MATs/music_mat_wl4_fma_large/148436...,Poder Fantasma,"['International', 'Latin America']",NaN


In [3]:
# modified from here: https://www.tensorflow.org/hub/tutorials/yamnet
# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
    """Returns list of class names corresponding to score vector."""
    class_names = []
    with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            class_names.append(row['display_name'])
    return class_names

class_map_path = model.class_map_path().numpy()
class_names = class_names_from_csv(class_map_path)

# verify and convert a loaded audio is on the proper sample_rate (16K), otherwise it would affect the model's results.
def ensure_sample_rate(original_sample_rate, waveform, desired_sample_rate=16000):
    """Resample waveform if required."""
    if original_sample_rate != desired_sample_rate:
        desired_length = int(round(float(len(waveform)) / original_sample_rate * desired_sample_rate))
        waveform = scipy.signal.resample(waveform, desired_length)
    return desired_sample_rate, waveform

def estimate_voice(df_meta, n_row):
    # st = time.time()
    path = 'yamnet/tensorFlow2/yamnet/1'
    model = tf.saved_model.load(path, tags=None, options=None)
    filename = df_meta['filepath'].iloc[n_row]
    frame_offset = df_meta['startPoint'].iloc[n_row]-1
    num_frame = df_meta['endPoint'].iloc[n_row]-frame_offset+1
    y_tensor, sr = torchaudio.load(filename, frame_offset=frame_offset, num_frames=num_frame)
    if y_tensor.shape[0]==1: # if the audio is mono, duplicate the channel
        y_tensor = torch.stack((y_tensor, y_tensor), dim=0)
    separator = demucs.api.Separator()
    origin, separated = separator.separate_tensor(wav=y_tensor, sr=sr) # demucs.api.Separator() by default will make sure the sampling rate is at 44.1 kHz
    _, waveform = ensure_sample_rate(44100, np.array(separated['vocals'].mean(axis=0))) # convert to sr=16000
    scores, _, _ = model(np.array(separated['vocals'].mean(axis=0))) # use YAMNET to score the audio waveform
    scores_np = scores.numpy()
    class_timeseries = scores_np.argmax(axis=1) # the max likelihood category per time-frame
    voice = np.any((class_timeseries >= 0) & (class_timeseries <= 35)) # whether the label of any time-frame belongs to any voice labels
    # infered_class = class_names[scores_np.mean(axis=0).argmax()]
    # voice = infered_class in class_names[0:35] # whether 
    # # get the execution time
    # et = time.time()
    # print('Execution time:', et - st, 'seconds')
    return voice